# Sample script to auto schedule work items

### Description

- This script enables users to auto schedule one or more work items.
- It takes a list of work item IDs and schedules the work items based on:
    1. The system that matches the system filter of the work item.
    2. The system that has the required number of fixtures to run the work item.
    3. The earliest available time slot considering the estimated duration of the work item within a range of six months from today.
- To specify the number of fixtures required to run a work item, users can add a custom property named `Number of fixtures` specifying the required fixture count. If this property is missing, invalid, or greater than 8, the script defaults the value to 1.

Note: To customize the scheduling algorithm, modify the [Actions](#Actions), [APIs](#APIs) and [Algorithm](#Algorithm)

## Imports

Import the necessary Python modules to execute the notebook. [**`nisystemlink.clients`**](https://github.com/ni/nisystemlink-clients-python) provides the predefined models and methods for `Work Items`, `System` and `Asset` APIs. **`Scrapbook`** is used to run notebooks and record data for integration with the SystemLink Notebook Execution Service. **`Datetime`** and **`Typing`** are also used.

In [ ]:
from datetime import datetime, timedelta, timezone
from typing import Dict, List, Tuple, Any, Optional
import scrapbook as sb

from nisystemlink.clients.assetmanagement import AssetManagementClient
from nisystemlink.clients.assetmanagement.models import (
    Asset,
    QueryAssetsRequest
)
from nisystemlink.clients.systems import SystemsClient
from nisystemlink.clients.systems.models import QuerySystemsRequest
from nisystemlink.clients.work_item import WorkItemClient
from nisystemlink.clients.work_item.models import (
    QueryWorkItemsRequest,
    ScheduleWorkItemsRequest,
    ScheduleWorkItemRequest,
    ScheduleDefinition,
    ScheduleResourcesDefinition,
    ScheduleResourceDefinition,
    ScheduleSystemResourceDefinition,
    ResourceSelectionDefinition,
    SystemResourceSelectionDefinition,
    WorkItem
)

## Parameters

- **`work_item_ids`**: IDs of the work items to be scheduled.

Parameters are also listed in the metadata for the **parameters cell**, along with their default values.  
The Notebook Execution services use that metadata to pass parameters to this notebook.  
To view the metadata:
- Select the code cell
- Click the **wrench icon** in the right panel.

### Sample metadata

```json
{
  "papermill": {
    "parameters": {
      "work_item_ids": []
    }
  },
  "systemlink": {
    "namespaces": [],
    "parameters": [
      {
        "display_name": "Work Item IDs",
        "id": "work_item_ids",
        "type": "string[]"
      }
    ],
    "version": 1
  },
  "tags": ["parameters"]
}
```
For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/lawork/usage-parameterize.html#how-parameters-work).

In [ ]:
# Add the work item IDs as a list of strings here
work_item_ids = []

## Constants

In [ ]:
# Max systems to be queried
MAX_SYSTEMS_COUNT = 1000 

# Max fixtures to be queried
MAX_FIXTURES_COUNT = 1000

# Max work items to be queried
MAX_WORK_ITEMS_COUNT = 10000

# Max work items to be queried per query
MAX_WORK_ITEMS_COUNT_PER_QUERY = 1000

# Default estimated duration of work item = 1 day"
DEFAULT_WORK_ITEM_DURATION_IN_SECONDS =  24 * 60 * 60

# Route for the schedule page in SystemLink
SCHEDULE_ROUTE = "labmanagement/schedule"

_current_time = datetime.now(timezone.utc)
_end_time_six_months = _current_time + timedelta(days = 182)

start_time = _current_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
due_date_time = _end_time_six_months.strftime("%Y-%m-%dT%H:%M:%S.000Z")

## Python clients

Initialize `WorkItemClient`, `SystemsClient` and `AssetManagementClient` to access the respective APIs.

In [ ]:
system_client = SystemsClient()
asset_client = AssetManagementClient()
work_item_client = WorkItemClient()

## APIs

Provides methods to query work items, schedule work items, query systems and query assets.

Get work item details by its ID.

In [ ]:
def get_work_item(work_item_to_schedule_id) -> Optional[WorkItem]:
    try:
        return work_item_client.get_work_item(work_item_to_schedule_id)
    except Exception as e:
        return None

Query systems that match the work item's system filter.

In [ ]:
def query_systems(system_filter) -> List[Dict[str, Any]]:
    request = QuerySystemsRequest(
        take = MAX_SYSTEMS_COUNT,
        projection = "new(id,alias)",
        orderBy = "lastUpdatedTimeStamp DESC",
        filter = system_filter
    )

    try:
        response = system_client.query_systems(request)
        return response.data
    except Exception as e:
        return []

Query fixtures linked to a system based on the system IDs.

In [ ]:
def query_fixtures(systems) -> List[Asset]:
    system_ids = ",".join(f'"{system["id"]}"' for system in systems)

    request = QueryAssetsRequest(
        take = MAX_FIXTURES_COUNT,
        filter = (
            'AssetType == "FIXTURE" && '
            f'(locations.Any(location => new[] {{{system_ids}}}.Contains(location.minionId)))'
        ),
        orderBy = "LAST_UPDATED_TIMESTAMP",
        descending = True,
    )

    try:
        response = asset_client.query_assets(request)
        return response.assets
    except Exception as e:
        return []

Query work items scheduled to systems across a timeframe (six months from today).

In [ ]:
def query_work_items(systems) -> Tuple[List[WorkItem], bool]:
    system_conditions = [f'resources.systems.selections.Any(s => s.id == "{system["id"]}")' for system in systems]

    if system_conditions:
        system_ids = f"({' or '.join(system_conditions)}) and "
    else:
        system_ids = ""

    work_items_filter = (
        f"{system_ids}"
        f"(schedule.plannedStartDateTime <= \"{due_date_time}\" and "
        f"schedule.plannedEndDateTime >= \"{start_time}\")"
    )

    all_work_items = []
    continuation_token = None

    while len(all_work_items) < MAX_WORK_ITEMS_COUNT:
        query_request = QueryWorkItemsRequest(
            filter = work_items_filter,
            take = MAX_WORK_ITEMS_COUNT_PER_QUERY,
            descending = True,
            continuation_token = continuation_token
        )

        try:
            response = work_item_client.query_work_items(query_request)
        except Exception as e:
            return [], False

        if response.work_items:
            all_work_items.extend(response.work_items)
            continuation_token = response.continuation_token

            if len(response.work_items) < MAX_WORK_ITEMS_COUNT_PER_QUERY or continuation_token is None:
                break
        else:
            break

    return all_work_items[:MAX_WORK_ITEMS_COUNT], True


Schedule a work item on a given system, fixtures and at a specific time slot and returns the schedule status.

In [ ]:

def _is_schedule_successful(schedule_response) -> bool:
    return schedule_response.failed_work_items is None

In [ ]:
def schedule_work_item(work_item_id, slot, fixture_ids) -> bool:
    fixture_resource = None
    if fixture_ids:
        fixture_resource = ScheduleResourceDefinition(
            selections = [
                ResourceSelectionDefinition(id = fixture_id) 
                for fixture_id in fixture_ids
            ]
        )
    
    request = ScheduleWorkItemsRequest(
        work_items = [
            ScheduleWorkItemRequest(
                id = work_item_id,
                schedule = ScheduleDefinition(
                    planned_start_date_time = slot['planned_start_time'],
                    planned_end_date_time = slot['planned_end_time'],
                    planned_duration_in_seconds = slot['planned_duration_in_seconds']
                ),
                resources = ScheduleResourcesDefinition(
                    systems = ScheduleSystemResourceDefinition(
                        selections = [
                            SystemResourceSelectionDefinition(
                                id = slot['system_id']
                            )
                        ]
                    ),
                    fixtures = fixture_resource
                )
            )
        ],
        replace = True
    )

    result = work_item_client.schedule_work_items(request)
    return _is_schedule_successful(result)


## Algorithm

The algorithm finds the earliest available time slot to schedule a work item that meets its resource requirements.

Convert fixtures and systems to dictionary (systemId as key and list of fixture ids as value).

In [ ]:
def get_system_fixtures_map(fixtures) -> Dict[str, List[str]]:
    system_fixtures_map = {}
    for fixture in fixtures:
        system_id = fixture.location.minion_id
        fixture_id = fixture.id
        
        if system_id not in system_fixtures_map:
            system_fixtures_map[system_id] = []
        
        system_fixtures_map[system_id].append(fixture_id)
    return system_fixtures_map

Convert work items and systems to dictionary (systemId as key and work item's planned start date time and estimated end date time as value).

In [ ]:
def organize_work_items_by_system(work_items, systems) -> Dict[str, WorkItem]:
    system_with_work_items = {}

    # Populate the dictionary with work items
    for work_item in work_items:
        if work_item.resources and work_item.resources.systems and work_item.resources.systems.selections:
            system_id = work_item.resources.systems.selections[0].id
            work_item_info = {
                'planned_start_time': work_item.schedule.planned_start_date_time,
                'planned_end_time': work_item.schedule.planned_end_date_time
            }
            if system_id not in system_with_work_items:
                system_with_work_items[system_id] = []
            system_with_work_items[system_id].append(work_item_info)

    # Ensure all systems are represented in the dictionary
    for system in systems:
        system_id = system['id']
        if system_id not in system_with_work_items:
            system_with_work_items[system_id] = []

    return system_with_work_items

Sort work items in the system by their estimated end times.

In [ ]:
def sort_and_filter_system_work_items(
    system_with_work_items: Dict[str, List[Dict[str, str]]]
) -> Dict[str, List[Dict[str, str]]]:

    current_date_time = datetime.fromisoformat(start_time.replace("Z", "+00:00"))
    
    for system_id, work_items in system_with_work_items.items():
        work_items.sort(key = lambda work_item: work_item['planned_end_time'])

        work_items = [
            work_item for work_item in work_items 
            if work_item['planned_end_time'] > current_date_time
        ]

        system_with_work_items[system_id] = work_items

    return system_with_work_items

Find available time slots to schedule the work item on all matching systems, and return the earliest one.

In [ ]:
def find_available_slots(
    system_with_work_items: Dict[str, List[Dict[str, str]]],
    work_item: WorkItem
) -> List[Dict[str, str]]:

    current_date_time = datetime.fromisoformat(start_time.replace("Z", "+00:00"))
    
    work_item_duration = timedelta(
        seconds = work_item.schedule.planned_duration_in_seconds
            if work_item.schedule and work_item.schedule.planned_duration_in_seconds != None else DEFAULT_WORK_ITEM_DURATION_IN_SECONDS
    )

    available_slots = []

    for system_id, occupied_slots in system_with_work_items.items():
        occupied_slots.sort(
            key = lambda occupied_slot: occupied_slot['planned_start_time']
        )
        
        last_busy_end_time = current_date_time

        for slot in occupied_slots:
            slot_start_time = slot['planned_start_time']
            slot_end_time = slot['planned_end_time']

            if slot_start_time > last_busy_end_time + work_item_duration:
                available_slots.append({
                    'system_id': system_id,
                    'planned_start_time': last_busy_end_time.isoformat(),
                    'planned_end_time': (last_busy_end_time + work_item_duration).isoformat(),
                    'planned_duration_in_seconds': int(work_item_duration.total_seconds())
                })
                break

            last_busy_end_time = max(last_busy_end_time, slot_end_time)

        if last_busy_end_time + work_item_duration < datetime.fromisoformat(due_date_time.replace("Z", "+00:00")):
            available_slots.append({
                'system_id': system_id,
                'planned_start_time': last_busy_end_time.isoformat(),
                'planned_end_time': (last_busy_end_time + work_item_duration).isoformat(),
                'planned_duration_in_seconds': int(work_item_duration.total_seconds())
            })

    available_slots.sort(key = lambda available_slot: available_slot['planned_start_time'])

    return available_slots

Get earliest available timeslots based on fixture availability in a system.

In [ ]:
def get_available_slot(
    available_slots,
    system_fixtures_map,
    number_of_fixtures_to_select
) -> Dict[str, Any]:

    if(number_of_fixtures_to_select == 0):
        return available_slots[0] if available_slots else {}
    
    for slot in available_slots:
        system_id = slot['system_id']
        fixture_ids = system_fixtures_map.get(system_id, [])
        
        if len(fixture_ids) >= number_of_fixtures_to_select:
            selected_fixture_ids = fixture_ids[:number_of_fixtures_to_select]
            slot['fixture_ids'] = selected_fixture_ids
            return slot
    
    return available_slots[0] if available_slots else {}

Take a list of time intervals (as tuples of datetime objects) and merge any overlapping intervals into non-overlapping ones.

In [ ]:
def merge_intervals(
    intervals: List[Tuple[datetime, datetime]]
) -> List[Tuple[datetime, datetime]]:

    if not intervals:
        return []

    intervals.sort()
    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        last_start, last_end = merged[-1]
        if current_start <= last_end:
            merged[-1] = (last_start, max(last_end, current_end))
        else:
            merged.append((current_start, current_end))
    
    return merged

Identify available time slots within a specified range by excluding the given busy intervals.

In [ ]:
def invert_intervals(
    intervals: List[Tuple[datetime, datetime]],
    start: datetime, end: datetime
) -> List[Tuple[datetime, datetime]]:

    available = []
    prev_end = start

    for busy_start, busy_end in intervals:
        if prev_end < busy_start:
            available.append((prev_end, busy_start))
        prev_end = max(prev_end, busy_end)

    if prev_end < end:
        available.append((prev_end, end))

    return available

Find the earliest time slot within a given time range where a work item can reserve a system and fixtures without conflicting with other work item schedules.

In [ ]:
def find_earliest_common_slot(
    fixtures: List[str],
    system: str,
    work_items,
    number_of_fixtures_to_schedule: int,
    duration: timedelta,
    search_start: datetime,
    search_end: datetime
) -> Dict[str, Any]:

    system_busy = []
    fixture_busy = {slave: [] for slave in fixtures}

    for work_item in work_items:
        planned_start_time = (
           work_item.schedule.planned_start_date_time
            if work_item.schedule and isinstance(work_item.schedule.planned_start_date_time, datetime)
           else datetime.fromisoformat(work_item.schedule.planned_start_date_time) if work_item.schedule else None
        )
        
        planned_end_time = (
            work_item.schedule.planned_end_date_time
            if work_item.schedule and isinstance(work_item.schedule.planned_end_date_time, datetime)
            else datetime.fromisoformat(work_item.schedule.planned_end_date_time) if work_item.schedule else None
        )

        system_id = None
        fixture_ids = []
        if work_item.resources:
            if work_item.resources.systems and work_item.resources.systems.selections:
                system_id = work_item.resources.systems.selections[0].id
            if work_item.resources.fixtures and work_item.resources.fixtures.selections:
                fixture_ids = [f.id for f in work_item.resources.fixtures.selections]

        if system_id == system and len(fixture_ids) == 0:
            system_busy.append((planned_start_time, planned_end_time))
        for fixture in fixture_ids:
            if fixture in fixture_busy:
                fixture_busy[fixture].append((planned_start_time, planned_end_time))

    system_busy = merge_intervals(system_busy)

    for fixture in fixtures:
        combined = fixture_busy[fixture] + system_busy
        fixture_busy[fixture] = merge_intervals(combined)

    fixture_available = {}
    for fixture in fixtures:
        fixture_available[fixture] = invert_intervals(
            fixture_busy[fixture],
            search_start,
            search_end
        )

    time_points = set()
    for intervals in fixture_available.values():
        for start, end in intervals:
            time_points.add(start)
            time_points.add(end)
    time_points = sorted(time_points)

    for time_point in time_points:
        available_fixtures = []
        for fixture, intervals in fixture_available.items():
            for interval_start, interval_end in intervals:
                if interval_start <= time_point and interval_end >= time_point + duration:
                    available_fixtures.append(fixture)
                    break

        if len(available_fixtures) >= number_of_fixtures_to_schedule:
            return {
                'planned_start_time': time_point,
                'planned_end_time': time_point + duration,
                'planned_duration_in_seconds': int(duration.total_seconds()),
                'fixture_ids': available_fixtures[:number_of_fixtures_to_schedule]
            }

    return {}

Find the earliest available time slot to schedule a work item across all matching systems and their fixtures.

In [ ]:
def find_best_system(
    system_fixture_map: Dict[str, List[str]],
    work_item, 
    number_of_fixtures_to_schedule: int,
    duration: timedelta,
    search_start: datetime,
    search_end: datetime
) -> Dict[str, Any]:

    best_option = None

    for system, fixtures in system_fixture_map.items():
        result = find_earliest_common_slot(
            fixtures,
            system,
            work_item,
            number_of_fixtures_to_schedule,
            duration,
            search_start,
            search_end
        )

        if result != {}:
            if not best_option or result['planned_start_time'] < best_option['planned_start_time']:
                best_option = {
                    'system_id': system,
                    'planned_start_time': result['planned_start_time'],
                    'planned_end_time': result['planned_end_time'],
                    'planned_duration_in_seconds': result['planned_duration_in_seconds'],
                    'fixture_ids': result['fixture_ids']
                }

    return best_option

## Actions

Validates the input and manages the execution flow.

Validate the number of work item IDs.

In [ ]:
def validate_work_item_ids(work_item_ids) -> bool:
    if len(work_item_ids) == 0:
        print("Required at least one work item id.")
        return False
    if len(work_item_ids) > 1000:
        print("Update limit exceeded: Only up to 1000 work items can be updated at a time.")
        sb.glue("Update limit exceeded: Only up to 1000 work items can be updated at a time.")
        return False
    return True

Get the number of fixtures to schedule.

In [ ]:
def get_number_of_fixtures(work_item) -> int:
    _number_of_fixtures = work_item.properties.get("Number of fixtures") if work_item.properties else None

    return int(_number_of_fixtures) if (
        isinstance(_number_of_fixtures, str)
        and _number_of_fixtures.isdigit()
        and (1 <= int(_number_of_fixtures) <= 8)
    ) else 1

Fetch systems and fixtures for the given work item.

In [ ]:
def fetch_systems_and_fixtures(work_item) -> Tuple[List[Dict[str, Any]], Dict[str, List[str]]]:
    system_filter = work_item.resources.systems.filter if work_item.resources and work_item.resources.systems else None
    systems = query_systems(system_filter)
    if systems is []:
        return [], {}
    fixtures = query_fixtures(systems)
    system_fixtures_map = get_system_fixtures_map(fixtures)
    return systems, system_fixtures_map

Find the system with fixture availability based on the earliest available timeslot and then schedule the work item.

In [ ]:
def find_and_schedule(
    work_item_to_schedule,
    systems,
    system_fixtures_map,
    work_items,
    number_of_fixtures_to_schedule,
    duration,
    start_date_time
) -> bool:

    # Try to find the best system.
    best_system = find_best_system(
        system_fixtures_map,
        work_items,
        number_of_fixtures_to_schedule,
        duration,
        start_date_time,
        datetime.fromisoformat(due_date_time.replace("Z", "+00:00"))
    )

    if best_system:
        fixtureIds = best_system['fixture_ids']
        best_system['planned_start_time'] = best_system['planned_start_time'].isoformat()
        best_system['planned_end_time'] = best_system['planned_end_time'].isoformat()
        return schedule_work_item(work_item_to_schedule.id, best_system, fixtureIds)

    # Fallback to available slot scheduling, as the required specifications are not met.
    system_with_work_items = organize_work_items_by_system(work_items, systems)
    sorted_system_work_items = sort_and_filter_system_work_items(system_with_work_items)
    available_slots_in_systems = find_available_slots(sorted_system_work_items, work_item_to_schedule)
    earliest_available_slot = get_available_slot(
        available_slots_in_systems,
        system_fixtures_map,
        number_of_fixtures_to_schedule
    )

    # Handle case when no slot is available
    if not earliest_available_slot:
        print("No available slot found for scheduling.")
        return False

    fixture_ids = system_fixtures_map.get(earliest_available_slot['system_id'], [])[:number_of_fixtures_to_schedule]

    return schedule_work_item(
        work_item_to_schedule.id,
        earliest_available_slot,
        fixture_ids
    )

Process a single work item from the list.

In [ ]:
scheduled_work_items = []
scheduled_work_item_names = []
unscheduled_work_items = []

def process_work_item(work_item_id, work_item_ids) -> None:
    work_item_to_schedule = get_work_item(work_item_id)
    if work_item_to_schedule is None:
        unscheduled_work_items.append(work_item_id)
        return False
    
    number_of_fixtures_to_schedule = get_number_of_fixtures(work_item_to_schedule)
    
    # If the number of fixtures is invalid, skip scheduling
    if number_of_fixtures_to_schedule > 8:
        unscheduled_work_items.append(work_item_id)
        return False

    # Fetch systems and fixtures
    systems, system_fixtures_map = fetch_systems_and_fixtures(work_item_to_schedule)
    if systems is []:
        unscheduled_work_items.append(work_item_id)
        return False

    # Query existing work items
    work_items, work_item_received = query_work_items(systems)
    if work_item_received is False:
        unscheduled_work_items.append(work_item_id)
        return False

    # Scheduling logic
    duration = timedelta(
        seconds = work_item_to_schedule.schedule.planned_duration_in_seconds if work_item_to_schedule.schedule and work_item_to_schedule.schedule.planned_duration_in_seconds else DEFAULT_WORK_ITEM_DURATION_IN_SECONDS
    )
        
    start_date_time = datetime.fromisoformat(start_time.replace("Z", "+00:00"))

    is_scheduled = find_and_schedule(
        work_item_to_schedule,
        systems,
        system_fixtures_map,
        work_items,
        number_of_fixtures_to_schedule,
        duration,
        start_date_time
    )
    
    if is_scheduled:
        scheduled_work_item_names.append(work_item_to_schedule.name)
        scheduled_work_items.append(work_item_id)
    else:
        unscheduled_work_items.append(work_item_id)

    return True


Schedule work items from the provided list of work item IDs.

In [ ]:
def schedule_work_items(work_item_ids) -> None:
    if not validate_work_item_ids(work_item_ids):
        return
    
    # Process each work item
    for work_item_id in work_item_ids:
        process_work_item(work_item_id, work_item_ids)

In [ ]:
schedule_work_items(work_item_ids)

## Executions page output

Send the output to the Execution Results page via Scrapbook. This includes:

1. The total number of work items processed.
1. A list of scheduled work item names.
1. A list of unscheduled work item names.
1. A link to the schedule page. The work item IDs of the updated work items included in this URL will be highlighted on the schedule page, making it easy to differentiate updated work items from others.

In [ ]:
work_item_names = ', '.join(scheduled_work_item_names)
sb.glue("Total work items", len(work_item_ids))
sb.glue("Scheduled work items", work_item_names)

if len(unscheduled_work_items) == 0:
    sb.glue("Unscheduled work items",  "-")
else:
    sb.glue("Unscheduled work items",  unscheduled_work_items)

if scheduled_work_items:
    scheduled_work_item_ids = ','.join(scheduled_work_items)
    sb.glue(
        "View scheduled work items in schedule page",
        f"<a href=\"../../{SCHEDULE_ROUTE}?work-items={scheduled_work_item_ids}\">Schedule View</a>"
    )

## Output

Print the script output.

In [ ]:
print("Total work items:", len(work_item_ids))
print("Scheduled work items:", work_item_names)
if len(unscheduled_work_items) == 0:
    print("Unscheduled work items: -")
else:
    print("Unscheduled work items:", unscheduled_work_items)

## Next Steps
Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as **`Work Item Scheduler`**.